# Week 11

I approached this problem in the following way:

1. Find all pairs with "distance 1" without limiting the number of movies

    (this operation does not take much time, but its effectiveness is great)
    

2. Go over the data after step 1 and mark all pairs that have a connecting actor as "distance 2"


3. Run function to try to find "distance 2" for all remaining cases

In [1]:
import requests
from bs4 import BeautifulSoup
import urllib
import re
import itertools
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt
import pandas as pd
from imdb_helper_functions import actor_name_function

In [2]:
def get_movies_by_actor_soup(actor_page_soup, num_of_movies_limit=None):
    #use regular expressions to get the full list of movies both odd/even
    soup_movie = actor_page_soup.find_all('div', attrs={"class": re.compile('^filmo-row')})
    result = []
    counter = 0
    
    for el in soup_movie:
        j = str(el)
        #filter movies as specified in the task
        if ((('actor' in j) is True or ('actress' in j) is True) 
            and ('in_production' in j) is False 
            and ('TV Series' in j) is False 
            and ('Short' in j) is False 
            and ('Video Game' in j) is False 
            and ('Video short' in j) is False 
            and ('Video' in j) is False 
            and ('TV Movie' in j) is False 
            and ('TV Mini Series' in j) is False 
            and ('TV Series short' in j) is False 
            and ('TV Special' in j) is False):
            #pairing a 'movie, link' list
            url_movie = urllib.parse.urljoin('https://www.imdb.com/', el.find('a')['href'])
            pair = (el.find('a').text, url_movie)
            result.append(pair)
            #implement a counter for num_of_movies_limit
            counter += 1
            if counter == num_of_movies_limit:
                break
    return result

In [3]:
def get_actors_by_movie_soup(cast_page_soup, num_of_actors_limit=None):
    #use regular expressions to get the full list of actors both odd/even
    soup_actor = cast_page_soup.find_all('tr', attrs={"class": re.compile('^')})
    result = []
    counter = 0
    
    #pairing a 'actor, link' list with using of actor_name_function (getting the name of the actor)
    for el in soup_actor:
        url_movie = urllib.parse.urljoin('https://www.imdb.com/', el.find('a')['href'])
        pair = (actor_name_function(el), url_movie)
        result.append(pair)
        #implement a counter for num_of_actors_limit
        counter += 1
        if counter == num_of_actors_limit:
            break
    return result

In [4]:
list_of_actors =  [['Dwayne Johnson', 'https://www.imdb.com/name/nm0425005/'], 
      ['Chris Hemsworth', 'https://www.imdb.com/name/nm1165110/'],
     ['Robert Downey Jr.', 'https://www.imdb.com/name/nm0000375/'],
     ['Akshay Kumar', 'https://www.imdb.com/name/nm0474774/'],
     ['Jackie Chan', 'https://www.imdb.com/name/nm0000329/'],
     ['Bradley Cooper', 'https://www.imdb.com/name/nm0177896/'],
     ['Adam Sandler', 'https://www.imdb.com/name/nm0001191/'],
     ['Scarlett Johansson', 'https://www.imdb.com/name/nm0424060'],
     ['Sofia Vergara', 'https://www.imdb.com/name/nm0005527/'],
     ['Chris Evans', 'https://www.imdb.com/name/nm0262635']]

In [5]:
list_of_actors1 =  [['Robert Downey Jr.', 'https://www.imdb.com/name/nm0000375/'],
     ['Akshay Kumar', 'https://www.imdb.com/name/nm0474774/'],
     ['Jackie Chan', 'https://www.imdb.com/name/nm0000329/'],
     ['Bradley Cooper', 'https://www.imdb.com/name/nm0177896/'],
     ['Adam Sandler', 'https://www.imdb.com/name/nm0001191/'],
     ['Scarlett Johansson', 'https://www.imdb.com/name/nm0424060'],
     ['Sofia Vergara', 'https://www.imdb.com/name/nm0005527/'],
     ['Chris Evans', 'https://www.imdb.com/name/nm0262635']]

In [15]:
def get_movie_distance(actor_start_url, actor_end_url, num_of_actors_limit=None, num_of_movies_limit=None):
    
    headers = {'Accept-Language': 'en',
                        'X-FORWARDED-FOR': '2.21.184.0'}
    
    actor_end_movies = []
    response = requests.get(actor_end_url, headers=headers)
    actor_page_soup = BeautifulSoup(response.text)
    movies_with_url_end = get_movies_by_actor_soup(actor_page_soup, num_of_movies_limit)
    for x in movies_with_url_end:
        actor_end_movies.append(x[0])
        
    list_of_movies_distance_2 = []
    list_of_movies_distance_2_2 = []
    list_of_movies_distance_3 = []

    response = requests.get(actor_start_url, headers=headers)
    actor_page_soup = BeautifulSoup(response.text)
    movies_with_url = get_movies_by_actor_soup(actor_page_soup, num_of_movies_limit)
    
    set_of_movies_distance_1 = set()
    for i in movies_with_url:
        set_of_movies_distance_1.add(i[0])
        
    for i in actor_end_movies:
        if i in set_of_movies_distance_1:
            return 1

    for movie in movies_with_url :
        
        response = requests.get(movie[1]+'fullcredits', headers=headers)
        cast_page_soup = BeautifulSoup(response.text)
        actors_with_url = get_actors_by_movie_soup(cast_page_soup, num_of_actors_limit)
        
        for actor in actors_with_url :
            
            response = requests.get(actor[1], headers=headers)
            actor_page_soup = BeautifulSoup(response.text)
            movies_with_url_2 = get_movies_by_actor_soup(actor_page_soup, num_of_movies_limit)
            list_of_movies_distance_2.append(movies_with_url_2)
    
    set_of_movies_distance_2 = set()
    for i in list_of_movies_distance_2:
        for j in i:
            set_of_movies_distance_2.add(j[0])
            list_of_movies_distance_2_2.append(j[1])
            
    for i in actor_end_movies:
        if i in set_of_movies_distance_2:
            return 2
    
    list_of_movies_distance_2_2 = list(set(list_of_movies_distance_2_2))
    
    for movie in list_of_movies_distance_2_2:
        
        response = requests.get(movie+'fullcredits', headers=headers)
        cast_page_soup = BeautifulSoup(response.text)
        actors_with_url = get_actors_by_movie_soup(cast_page_soup, num_of_actors_limit)
        
        for actor in actors_with_url :
            
            response = requests.get(actor[1], headers=headers)
            actor_page_soup = BeautifulSoup(response.text)
            movies_with_url_3 = get_movies_by_actor_soup(actor_page_soup, num_of_movies_limit)
            list_of_movies_distance_3.append(movies_with_url_3)

    set_of_movies_distance_3 = set()
    for i in list_of_movies_distance_3:
        for j in i:
            set_of_movies_distance_3.add(j[0])
            
    for i in actor_end_movies:
        if i in set_of_movies_distance_3:
            return 3

In [16]:
def get_movie_descriptions_by_actor_soup(actor_page_soup):
    
    headers = {'Accept-Language': 'en',
          'X-FORWARDED-FOR': '2.21.184.0'}
    
    #use regular expressions to get the full list of movies both odd/even
    soup_movie = actor_page_soup.find_all('div', attrs={"class": re.compile('^filmo-row')})
    result = []
    
    for el in soup_movie:
        j = str(el)
        
        #filter movies as specified in the task
        if ((('actor' in j) is True or ('actress' in j) is True) 
            and ('in_production' in j) is False 
            and ('TV Series' in j) is False 
            and ('Short' in j) is False 
            and ('Video Game' in j) is False 
            and ('Video short' in j) is False 
            and ('Video' in j) is False 
            and ('TV Movie' in j) is False 
            and ('TV Mini Series' in j) is False 
            and ('TV Series short' in j) is False 
            and ('TV Special' in j) is False):
            
            url_movie = urllib.parse.urljoin('https://www.imdb.com/', el.find('a')['href'])
            response_movie = requests.get(url_movie, headers=headers)
            soup = BeautifulSoup(response_movie.text)
            description = soup.find('span', attrs={'data-testid': 'plot-xl'}).text
            
            result.append(description)

    return result

In [17]:
list_of_actors =  [['Dwayne Johnson', 'https://www.imdb.com/name/nm0425005/'], 
      ['Chris Hemsworth', 'https://www.imdb.com/name/nm1165110/'],
     ['Robert Downey Jr.', 'https://www.imdb.com/name/nm0000375/'],
     ['Akshay Kumar', 'https://www.imdb.com/name/nm0474774/'],
     ['Jackie Chan', 'https://www.imdb.com/name/nm0000329/'],
     ['Bradley Cooper', 'https://www.imdb.com/name/nm0177896/'],
     ['Adam Sandler', 'https://www.imdb.com/name/nm0001191/'],
     ['Scarlett Johansson', 'https://www.imdb.com/name/nm0424060'],
     ['Sofia Vergara', 'https://www.imdb.com/name/nm0005527/'],
     ['Chris Evans', 'https://www.imdb.com/name/nm0262635']]

In [18]:
def save_to_files(list_of_actors):

    for actor in list_of_actors:

        f = open((str(actor[0]) + '.txt'), 'w')

        headers = {'Accept-Language': 'en',
                'X-FORWARDED-FOR': '2.21.184.0'}
        response = requests.get(actor[1], headers=headers)
        actor_page_soup = BeautifulSoup(response.text)
        x = get_movie_descriptions_by_actor_soup(actor_page_soup)

        for item in x:
            f.write('%s\n' % item)

        f.close()